In [1]:
import mpmath as mp

from code.python.mpmath_integration import quad_phi

In [251]:
x = mp.mpf('3')
alpha = mp.mpf('5')
beta = mp.mpf('1/2')
mu = mp.mpf('1')
delta = mp.mpf('4')

xmu = x - mu

gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

### 1. Expansion a -> 0

In [252]:
a = x-mu
b = -beta

In [119]:
def normcdf(x):
    return mp.erfc(-x / mp.sqrt(mp.mpf('2'))) / mp.mpf('2')

In [120]:
t = 1

normcdf(a / mp.sqrt(t) + b * mp.sqrt(t))

mpf('0.3445782583896758332631193239780179606992522743091937018131776106255297646403867195404139332445104985206')

In [121]:
normcdf(b * mp.sqrt(t))

mpf('0.3085375387259868963622953893916622601163978244454220631792238573208442045937204559747589399537550800944')

In [122]:
A = mp.exp(-b**2/2 * t) / mp.sqrt(2*mp.pi) / mp.sqrt(t)

In [123]:
s1 =  a
s2 = -a**2 * b / 2
s3 =  a**3 * (b**2 * t - 1) / t / 6
s4 = -a**4 * (b**3 * t - 3 * b) / t / 24
s5 =  a**5 * (b**4 * t**2 - 6 * b**2 * t + 3) / t**2 / 120
s6 = -a**6 * (b**5 * t**2 - 10 * b**3 * t + 15 * b) / t**2 / 720

S = s1 + s2 + s3 + s4 + s5 + s6

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.344578258425790132005014849032996501820701629426797153394846435765077449213588599427492802564101301052')

In [124]:
s1, s2, s3, s4, s5, s6

(mpf('0.1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002'),
 mpf('0.002499999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
 mpf('-0.0001250000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003'),
 mpf('-0.000005729166666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666716'),
 mpf('0.0000001302083333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333331'),
 mpf('0.000000008723958333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333452'))

In [125]:
print(
    1,
    b,
    (b**2 * t - 1) / t,
    (b**3 * t - 3 * b) / t,
    (b**4 * t**2 - 6 * b**2 * t + 3) / t**2,
    (b**5 * t**2 - 10 * b**3 * t + 15 * b) / t**2
)

1 -0.5 -0.75 1.375 1.5625 -6.28125


In [126]:
def P(k, t, b):
    s = 0
    for m in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** m / mp.factorial(m) / mp.factorial(k - 2 * m) * b ** (k - 2*m) / 2 ** m / t ** m

    return mp.factorial(k) * s

In [127]:
k = 0
P(k, t, b)

mpf('1.0')

In [128]:
K = 10
S = 0
for k in range(1, K):
    S += (-1) ** (k + 1) * a ** k / mp.factorial(k) * P(k-1, t, b)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.3445782583896726633126213841818708057364352384490663015586898866174163079066697746384902353138139675133')

Hermite probabilistic

In [14]:
def hermite_prob(k, z):
    s = 0
    for m in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** m / mp.factorial(m) / mp.factorial(k - 2 * m) * z ** (k - 2*m) / 2 ** m

    return mp.factorial(k) * s    

In [15]:
k = 10

P(k, t, b)

mpf('1215.9999999999998')

In [16]:
hermite_prob(k, b * mp.sqrt(t)) / t ** (k / 2)

mpf('1215.9999999999998')

In [17]:
mp.hermite(k, b * mp.sqrt(t / 2)) / (2 * t) ** (k / 2)

mpf('1216.0')

In [18]:
K = 40
S = 0
for k in range(1, K):
    S += (-1) ** (k + 1) * a ** k / mp.factorial(k) * mp.hermite(k-1, b * mp.sqrt(t / 2)) / (2 * t) ** ((k-1) / 2)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.22662735237686826')

In [19]:
K = 40
S = 0
for k in range(K):
    S += (-1) ** k * a ** (k + 1) / mp.factorial(k + 1) * mp.hermite(k, b * mp.sqrt(t / 2)) / (2 * t) ** (k / 2)

normcdf(b * mp.sqrt(t)) + A * S

mpf('0.22662735237686826')

### 2. Expansion

In [256]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.9599775405494657360087526092654300842676567924404618741762742877734998432533542621944042950278597682396')

In [21]:
quad_phi(mu, alpha, beta, mu, delta, digits=100)

mpf('0.10695839610478352')

In [22]:
C = delta * mp.exp(delta * gamma) / (2 * mp.pi)

In [23]:
mp.mp.dps = 100

N = 50
s = 0
for k in range(N):
    r = (-1) ** k * xmu ** (k + 1) / mp.factorial(k + 1) / 2 ** (k / 2)
    q = mp.quad(lambda t: t ** (-k/2 - 2) * mp.hermite(k, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    s += r * q

C * s

mpf('0.04347641340394729569951899595427667702656381936911577126182501627550004390562570760561472695291864090097')

In [24]:
r1 = quad_phi(mu, alpha, beta, mu, delta, digits=100) + C * s

In [25]:
r1, mp_result

(mpf('0.1504348095087308141162842752414471180429115980940278954755316183925193301234076023129822373200153345192'),
 mpf('0.1504348095087308052347107896333173618777248414806567845400163882616454210282536197036370400375892072737'))

##### Coefficients recursion

Ik+1

In [ ]:
k = 1
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * mp.hermite(k+1, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

In [ ]:
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (2 * b * mp.sqrt(t / 2) * mp.hermite(k, b * mp.sqrt(t / 2)) -2*k *  mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

Ik

In [ ]:
Ik = mp.quad(lambda t: t ** (-k/2 - 2) * mp.hermite(k, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
Ik, Ik * b * mp.sqrt(2)

Ik-1

In [ ]:
-2*k * mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

In [ ]:
mp.quad(lambda t: t ** (-(k+1)/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

In [ ]:
z = alpha ** 2 / 2
r = delta / alpha

In [ ]:
I11 = mp.quad(lambda t: t ** (-(k+1)/2 - 2 + 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

In [ ]:
I12 = mp.quad(lambda t: t ** (-(k+1)/2 - 2 + 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2))) * 1/z * z * (r**2/t**2 - 1) * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

In [ ]:
(I11 + I12) / r ** 2

In [ ]:
I11, I12

In [ ]:
mp.quad(lambda t: t ** (-k/2 + 3/2 - 2) * (mp.hermite(k-1, b * mp.sqrt(t / 2)))  * mp.exp(-z * (t + r ** 2 / t)), [0, mp.inf])

##### Coefficient finite series

In [ ]:
k = 3
mp.quad(lambda t: t ** (-k/2 - 2) * mp.hermite(k, b * mp.sqrt(t / 2)) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

In [ ]:
s = 0
for j in range(int(mp.floor(k/2)) + 1):
    s += (-1) ** j / mp.factorial(j) / mp.factorial(k - 2 * j) * (alpha / delta / beta ** 2 / 2) ** j * mp.besselk(j+1, alpha * delta)

2 * mp.factorial(k) * s * (alpha / delta) * (b * mp.sqrt(2)) ** k 

In [ ]:
j = 4
mp.quad(lambda t: t ** (-j - 2) * mp.exp(-delta ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])

In [ ]:
2 * (alpha / delta) ** (j + 1) * mp.besselk(j+1, alpha * delta)

#### Final expansion

In [282]:
N = 66

In [283]:
def Ak(k):
    s = 0
    for j in range(int(mp.floor(k/2)) + 1):
        s += (-1) ** j / mp.factorial(j) / mp.factorial(k - 2 * j) * (alpha / 2 / delta / beta ** 2) ** j * mp.besselk(j+1, alpha * delta)
    return s

In [284]:
s = 0
for k in range(N):
    s += (beta * (x-mu)) ** k / (k + 1) * Ak(k)

quad_phi(mu, alpha, beta, mu, delta, digits=100) + mp.exp(delta * gamma) * (x-mu) * alpha / mp.pi * s 

mpf('0.9599775405494658225921218489771005650012090018135787408930835607933424531844137210166518493466611648064')

In [285]:
r2 = quad_phi(mu, alpha, beta, mu, delta, digits=100) + mp.exp(delta * gamma) * (x-mu) * alpha / mp.pi * s 

In [286]:
r2, mp_result

(mpf('0.9599775405494658225921218489771005650012090018135787408930835607933424531844137210166518493466611648064'),
 mpf('0.9599775405494657360087526092654300842676567924404618741762742877734998432533542621944042950278597682396'))

In [287]:
float(abs(r2 / mp_result - 1))

9.01931197162734e-17